<div class="alert alert-warning">
    <strong>Analyst Note: Info</strong><br />
    
### READ THIS!
    
This notebook includes yellow **Analysis Notes** like this one, which provide instructions for the CCBB analyst performing this analysis.  Each note is coded to indicate what the analyst should do at that note:

* *Prepare*: Perform the steps specified in the note outside the notebook, then delete the note
* *Fill In*: Replace the note with a markdown cell containing text specific to this analysis
* *Modify Code*: Follow the instructions in the note to modify adjacent code, then delete the note
* *Check*: Check the info specified in the note, then delete the note
* *Info*: Read the note and make sure you understand it, then delete the note
* *Placeholder*: Decide if you can provide the requested info to improve the notebook; if so, add it and contact Amanda to get it checked into the git repository!  If not, just delete the note

<br />

**REMOVE OR REPLACE ALL ANALYST NOTES BEFORE DELIVERY TO CUSTOMER.** 

<br />
The final delivered notebook should not include any yellow cells.

</div>

<div class="alert alert-warning">
    <strong>Analyst Note: Fill In</strong><br />
    Fill in the human-readable name of your project as a header, such as:
    
   > # Dr. Doe Human Patient Time-Series
</div>

# RNASeq Enrichment Analyses

<div class="alert alert-warning">
    <strong>Analyst Note: Fill In</strong><br />
    Fill in the author attributions for your analysis, such as:
    
   > * Amanda Birmingham, CCBB (abirmingham@ucsd.edu)
   > * Based on upstream analysis by Guorong Xu, CCBB (g1xu@ucsd.edu)
</div>

* Modeled on "RNA-seq analysis is easy as 1-2-3 with limma, Glimma and edgeR" ([1](#Citations))

## Table of Contents
* [Introduction](#Introduction)
* [Parameter Input](#Parameter-Input)
* [Library Import](#Library-Import)
* [Enrichment Tool Set-Up](#Enrichment-Tool-Set-Up)
* [Enrichment Analysis Background](#Enrichment-Analysis-Background)
* [Enrichment Tool Set-Up](#Enrichment-Tool-Set-Up)
    * [WebGestalt](#WebGestalt)
    * [Pathview](#Pathview)
    * [Signaling Pathway Impact Analysis](#SPIA)
    * [Gene Set Variation Analysis](#GSVA)
* [Examination of Individual Comparisons](#Examination-of-Individual-Comparisons)

<div class="alert alert-warning">
    <strong>Analyst Note: Fill In</strong><br />
    Fill in table of contents links to the various comparions for your analysis, such as:
    
   > * [Day15vsDay0](#Day15vsDay0)
   > * [Day25vsDay0](#Day25vsDay0)
   > * [Day25vsDay15](#Day25vsDay15)
   
</div>

* [Understanding the Output](#Understanding-the-Output)
* [Summary](#Summary)
* [Citations](#Citations)
* [Appendix: R Session Info](#Appendix:-R-Session-Info)


## Introduction

This notebook takes in RNASeq data prepared by the previous notebook, "RNASeq_Differential_Expression_Testing", and uses the WebGestalt ([2](#Citations)) R ([3](#Citations)) package to test for enrichment of differentially expressed genes in annotated functions, pathways, and diseases.  It also uses the PathView [(4)](#Citations) BioConductor [(5)](#Citations) tool to color KEGG [(6)](#Citations) pathways by differential expression findings.

[Table of Contents](#Table-of-Contents)

## Parameter Input

<div class="alert alert-warning">
<h4>Analyst note: Modify Code</h4>
The values shown below are example settings, and should be overwritten with appropriate values for your analysis.
</div>

In [1]:
gPreviousStepRdataFp = "../interim/20190307_doe_human_patient_timeseries_differential_expression_20191018153313.RData"

<div class="alert alert-warning">
<h4>Analyst note: Info</h4>
The gSourceDir value shown below is a standard settings, and should NOT be changed without a clear reason and understanding of the consequences.
</div>

In [2]:
gSourceDir = "./src/"
# Import shared source code to load and save previous notebooks' environments:
source(paste0(gSourceDir, "ChainedNotebookSupport.R"))
# Import the previous step's state:
gPreviousStepEnv = loadToEnvironment(gPreviousStepRdataFp)

In [3]:
# Load relevant variables:
gProjectName = gPreviousStepEnv$gProjectName
gInterimDir = gPreviousStepEnv$gInterimDir
gOutputDir = gPreviousStepEnv$gOutputDir
gModeratedVoomedFitForContrastsWithLogFcThreshold = gPreviousStepEnv$gModeratedVoomedFitForContrastsWithLogFcThreshold

Populate the run name parameter automatically to ensure that outputs from different runs do not overwrite each other:

In [4]:
gRunName = makeRunName(gProjectName, "enrichment_analyses")
gRunName

[1] "20190307_doe_human_patient_timeseries_enrichment_analyses_20191101143736"

[Table of Contents](#Table-of-Contents)

## Library Import

Import the necessary R libraries for the analysis:

In [5]:
#install.packages("WebGestaltR")

In [6]:
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

In [7]:
#BiocManager::install("pathview", version = "3.8")

In [8]:
#BiocManager::install("SPIA", version = "3.8")

In [9]:
#BiocManager::install("GSVA", version = "3.8")
#BiocManager::install("GSEABase", version = "3.8")
#BiocManager::install("GSVAdata", version = "3.8")
#BiocManager::install("Biobase", version = "3.8")
#BiocManager::install("genefilter", version = "3.8")

In [10]:
library(limma)

In [11]:
library(WebGestaltR)

******************************************

*                                        *

*          Welcome to WebGestaltR !      *

*                                        *

******************************************



In [12]:
library(pathview)

Loading required package: org.Hs.eg.db
Loading required package: AnnotationDbi
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Lo

In [13]:
library(SPIA)

Loading required package: KEGGgraph

Attaching package: ‘KEGGgraph’

The following object is masked from ‘package:graphics’:

    plot



In [14]:
library(GSVA)
library(GSEABase)
library(GSVAdata)
library(Biobase)
library(genefilter)
data(c2BroadSets)

Loading required package: annotate
Loading required package: XML
Loading required package: graph

Attaching package: ‘graph’

The following object is masked from ‘package:XML’:

    addNode



ERROR: Error in library(GSVAdata): there is no package called ‘GSVAdata’


In [47]:
library(GSVAdata)

Loading required package: hgu95a.db



In [49]:
data(c2BroadSets)

In [15]:
# This line prevents SVG output, which interacts badly with export to HTML
options(jupyter.plot_mimetypes = c("text/plain", "image/png" ))

## Enrichment Analysis Background

<div class="alert alert-warning">
<h4>Analyst note: Placeholder</h4>

`TODO: May want to add description from/reference to Backes C, Keller A, Kuentzer J, Kneissl B, Comtesse N, Elnakady YA, Müller R,  Meese E, Lenhof HP. GeneTrail--advanced gene set enrichment analysis. Nucleic Acids Res. 2007 Jul;35(Web Server issue):W186-92.  which has good explanation of varieties of enrichment analysis.`

</div>

"Enrichment analysis" is an umbrella term for a class of analyses that investigate whether a gene set of interest (usually those found to be significant in an upstream analysis such as differential expression testing) are found to be "enriched" an annotated gene set (such as the set of genes in a particular pathway).  Enrichment can be determined in a number of different ways, with the most common being over-representation analysis (ORA) and gene set enrichment analysis (GSEA).  

Over-representation analysis simply compares gene lists, determining whether the genes in the gene set of interest are present in the annotated gene set in a significantly higher number (or a significantly lower number) than one would statistically expect by chance given the size and content of the two sets and the reference set from which they are drawn.  In this sort of analyis, the statistical validity of the outcome depends critically on the choice of correct reference set, which represents the set of genes from which the genes of interest can be drawn; while it is easy (and tempting) to use all the genes in the organism being examined as the reference set, this is almost certainly an over-estimate--which will lead to over-estimates of the significance of any enrichment found--and thus should be replaced with a more realistic number such as the number of genes actually detectable in the RNASeq experiment.

Conversely, gene set enrichment analysis makes use of not only the identities of the genes of interest but also their scores (such their differential expression fold-changes): the genes of interest are ranked by their scores, and the GSEA algorithm ([7](#Citations)) determines whether the subset of genes of interest that are also part of the chosen annotated set (e.g., a particular pathway) are localized toward the top or the bottom of this ranked list.  For example, if genes from a particular pathway are found to lie preferentially near the top of a list of genes differentially expressed in treatment versus control that has been ranked by decreasing fold-change, then this suggests that this pathway is UP-regulated in the treatment compared to the control.

Enrichment analyses such as ORA and GSEA can be performed using a wide range of different annotated sets. The most commonly used is the Gene Ontology (GO; [8](#Citation)), in which case the resulting analysis is often referred to as a "functional enrichment analysis" since it examines the enrichment of particular biological functions (as annotated in the GO) in the gene set of interest.  As discussed earlier, enrichment analysis can also be performed using pathways (from sources such as KEGG ([6](#Citation)) as the annotated gene set, in which case it may be called "pathway enrichment analysis".

## Enrichment Tool Set-Up

### WebGestalt  

While a large number of different enrichment analysis tools and techniques exist, we choose to use the WebGestalt package, which offers both ORA and GSEA algorithms and provides built-in access to a large number of annotated gene sets of interest.

<div class="alert alert-warning">
<h4>Analyst note: Modify Code</h4>
Set the appropriate organism for your analysis.
</div>

In [16]:
# organism must be one of the following set:
# athaliana
# btaurus
# celegans
# cfamiliaris
# dmelanogaster
# drerio
# ggallus
# hsapiens
# mmusculus
# rnorvegicus
# scerevisiae
# sscrofa
# others

gWebgestaltOrganism = "hsapiens"

WebGestalt offers a large variety of annotated dataset resources, grouped into 9 categories: geneontology, pathway, network, disease, drug, phenotype, chromosomalLocation, community-contributed, and other.  The resources available differ depending on the organism to be analyzed.  View the full list of resources available for the organism of interest:

In [17]:
listGeneSet(organism=gWebgestaltOrganism,hostName="http://www.webgestalt.org/")

name,description,idType
<chr>,<chr>,<chr>
geneontology_Biological_Process,The gene ontology biological process database was downloaded from http://www.geneontology.org/.,entrezgene
geneontology_Biological_Process_noRedundant,"The gene ontology biological process database was downloaded from http://www.geneontology.org/. Then, we only contain the non-redundant categories by selecting the most general categories in each branch of the GO DAG structure from all categories with the number of annotated genes from 20 to 500.",entrezgene
geneontology_Cellular_Component,The gene ontology cellular component database was downloaded from http://www.geneontology.org/.,entrezgene
geneontology_Cellular_Component_noRedundant,"The gene ontology cellular component database was downloaded from http://www.geneontology.org/. Then, we only contain the non-redundant categories by selecting the most general categories in each branch of the GO DAG structure from all categories with the number of annotated genes from 20 to 500.",entrezgene
geneontology_Molecular_Function,The gene ontology molecular function database was downloaded from http://www.geneontology.org/.,entrezgene
geneontology_Molecular_Function_noRedundant,"The gene ontology molecular function database was downloaded from http://www.geneontology.org/. Then, we only contain the non-redundant categories by selecting the most general categories in each branch of the GO DAG structure from all categories with the number of annotated genes from 20 to 500.",entrezgene
pathway_KEGG,The KEGG pathway database was downloaded from http://www.kegg.jp/.,entrezgene
pathway_Panther,The PANTHER pathway database was downloaded from http://www.pantherdb.org/pathway/.,entrezgene
pathway_Reactome,The Reactome pathway database was downloaded from http://www.reactome.org/.,entrezgene


<div class="alert alert-warning">
<h4>Analyst note: Prepare</h4>

**Decide which WebGestalt datasets to test for enrichment**.  Keep in mind that:

* the optimal set of datasets depends on both the organism under test and the hypothesis being tested
* the analyses--especially the GSEA analyses--take from several minutes to 10s of minutes each, so the list of datasets should be minimized
* the analysis of each dataset produces as lot of output to examine

    
For **HUMAN** analyses, selecting **a subset** of the list of datasets below is usually reasonable:

    'geneontology_Biological_Process','geneontology_Molecular_Function',
    'geneontology_Biological_Process_noRedundant','pathway_KEGG',
    'pathway_Panther','pathway_Reactome','pathway_Wikipathway',
    'network_Kinase_target','network_miRNA_target','network_PPI_BIOGRID',
    'network_Transcription_Factor_target','disease_Disgenet', 
    'disease_OMIM', 'drug_DrugBank', 'phenotype_Human_Phenotype_Ontology'
    
</div>

<div class="alert alert-warning">
<h4>Analyst note: Fill In</h4>
    
Replace this note with a description of which WebGestalt datasets were chosen for enrichment testing, such as:

> We choose to examine the Biological Process and Molecular Function domains of the Gene Ontology, as well as four pathway sets (KEGG, Panther, Reactome, and Wikipathway), four networks (kinase targets, miRNA targets, PPI BioGRID, and transcription factor targets), two disease sets (Disgenet and OMIM), DrugBank, and the Human Phenotype Ontology.  These will give us a broad overview of the involvement of the genes of interest in a wide variety of domains.

</div>

<div class="alert alert-warning">
    <strong>Analyst Note: Modify Code</strong><br />

Populate the `gWebgestaltDbs` variable below with the names of the WebGestalt datasets to be tested. 

Here is an example of what the `gWebgestaltDbs` definition could look like:

        # DO **NOT** JUST USE THIS COMMAND!
        # REPLACE IT WITH APPROPRIATE DATASET NAMES
        # STEPS FOR YOUR PROJECT!     

        gWebgestaltDbs = c('pathway_Panther','disease_OMIM', 'drug_DrugBank')

   
</div>

In [18]:
gWebgestaltDbs = c('pathway_Panther','disease_OMIM', 'drug_DrugBank')

In [34]:
runWebgestaltOra = function(organism, runName, outputDirectory, databases, fit, coefficientName, maxAdjPVal=0.05){
    tryCatch({
        top <- topTreat(fit, coef=coefficientName, n=Inf, sort.by='p')
        top$ENTREZ<-unlist(as.list(top$ENTREZID))
        top<-top[!is.na(top$ENTREZ),]
        top<-top[!duplicated(top$ENTREZ),]
        tg1<-top[top$adj.P.Val<maxAdjPVal,]

        DE_genes=tg1$ENTREZ
        ALL_genes=top$ENTREZ
        interestGeneFile = file.path(outputDirectory, sprintf('%s_DE_Genes_Webgestalt_ORA_input_%s.txt', runName, coefficientName))
        referenceGeneFile = file.path(outputDirectory, sprintf('%s_ReferenceGenes_Webgestalt_ORA_%s.txt', runName, coefficientName))
        write.table(DE_genes, interestGeneFile, sep='\t', row.names=F, col.names=F, quote=FALSE)
        write.table(ALL_genes, referenceGeneFile, sep='\t',row.names=F, col.names=F, quote=FALSE)

        for (database in databases){
            enrichResult <-WebGestaltR(enrichMethod='ORA', 
                                       organism=organism,
                                       enrichDatabase=database,
                                       interestGeneFile=interestGeneFile,
                                       interestGeneType='entrezgene',
                                       referenceGeneFile=referenceGeneFile,
                                       referenceGeneType='entrezgene',
                                       isOutput=TRUE,
                                       outputDirectory=outputDirectory,
                                       projectName=sprintf('%s_Webgestalt_ORA_%s_%s', runName, coefficientName, database))
        }
    }, error=function(e){print(paste0("WebGestaltR ORA failed for maxAdjPval <",maxAdjPVal," with message:", e))})
}

runWebgestaltGsea = function(organism, runName, outputDirectory, databases, fit, coefficientName, maxAdjPVal=0.05){
    tryCatch({
        top <- topTreat(fit, coef=coefficientName, n=Inf, sort.by='p')
        top$ENTREZ<-unlist(as.list(top$ENTREZID))
        top<-top[!is.na(top$ENTREZ),]
        top<-top[!duplicated(top$ENTREZ),]
        tg1<-top[top$adj.P.Val<maxAdjPVal,]

        DE_genesAndScores= data.frame(tg1$ENTREZ,tg1$logFC)
        interestGeneFile = file.path(outputDirectory, sprintf('%s_DE_GenesAndScores_Webgestalt_GSEA_input_%s.rnk', runName, coefficientName))
        write.table(DE_genesAndScores, interestGeneFile, sep='\t', row.names=F, col.names=F, quote=FALSE)

        for (database in databases){
            enrichResult<-WebGestaltR(enrichMethod="GSEA",
                                      organism=organism,
                                      enrichDatabase=database,
                                      interestGeneFile=interestGeneFile,
                                      interestGeneType='entrezgene', 
                                      collapseMethod="mean",
                                      isOutput=TRUE,
                                      outputDirectory=outputDirectory,
                                      projectName=sprintf('%s_Webgestalt_GSEA_%s_%s', runName, coefficientName, database))
        }
    }, error=function(e){print(paste0("WebGestaltR GSEA failed for maxAdjPval <",maxAdjPVal," with message:", e))})
}

In [35]:
globalRunWebgestalt = function(coefficientName, maxAdjPVal=0.05){
    runWebgestaltOra(gWebgestaltOrganism, gRunName, gOutputDir, gWebgestaltDbs,
                     gModeratedVoomedFitForContrastsWithLogFcThreshold, 
                     coefficientName, maxAdjPVal=maxAdjPVal)
    
    runWebgestaltGsea(gWebgestaltOrganism, gRunName, gOutputDir, gWebgestaltDbs,
                     gModeratedVoomedFitForContrastsWithLogFcThreshold, 
                     coefficientName, maxAdjPVal=maxAdjPVal)    
}

<div class="alert alert-warning">
    <strong>Analyst Note: Modify Code</strong><br />

The Pathview module below is useful ONLY if:

* The data being examined are from human or mouse
* It makes sense to visualize differentially expressed genes on a (small) list of known KEGG pathways 

**If you decide not to use Pathview, delete the entire "Pathview" subsection below**.
</div>

### Pathview
Pathview ([4](#Citation)) visualizes user-selected KEGG pathways with color coded log-fold changes from a given comparison. 

<div class="alert alert-warning">
    <strong>Analyst Note: Prepare</strong><br />   
    
Pathview requires a list of KEGG pathways of interest as input. Visit [https://www.kegg.jp/kegg/pathway.html](https://www.kegg.jp/kegg/pathway.html) , **identify the ids of pathways relevent to this experiment**, and determine the  **numeric portion** (only) of those ids.  For example, to examine the "Longevity regulating pathway", which has id `map04211`, use `04211`.

</div>

<div class="alert alert-warning">
    <strong>Analyst Note: Fill In</strong><br />   
    

Provide text indicating the names of the pathways entered and why they were selected.
</div>

<div class="alert alert-warning">
    <strong>Analyst Note: Modify Code</strong><br />

Populate the `gKeggPathwaysOfInterest` variable below with the ids of the KEGG pathways to be visualized in Pathview. 

Here is an example of what the `gKeggPathwaysOfInterest` definition could look like:

        # DO **NOT** JUST USE THIS COMMAND!
        # REPLACE IT WITH APPROPRIATE KEGG PATHWAY IDS
        # STEPS FOR YOUR PROJECT!     

        gKeggPathwaysOfInterest = c("04512", "00600", "04910", "04211")

   
</div>

In [36]:
gKeggPathwaysOfInterest = c("04512", "00600", "04910", "04211")

<div class="alert alert-warning">
    <strong>Analyst Note: Modify Code</strong><br />
    
Set the appropriate organism name recognized by Pathview in the code cell below.

**Note that the only available choices are "human" or "mouse"**.

</div>

In [37]:
gPathviewOrganismName = "human"

In [39]:
# Function for pathview - visualizes KEGG pathways
# species = "mouse" or "human"
runPathview = function(efitObject, outputDir, pathwayList, species, coefficientName){
    workingDir = getwd()
    
    # Create folder for Pathview files
    pathviewOutputDir = file.path(outputDir, paste0("Pathview_", coefficientName))    
    
    tryCatch({
        dir.create(pathviewOutputDir)
        setwd(pathviewOutputDir)

        # Extract DE genes and stats from efit object
        coef_table <- topTreat(efitObject, coef=coefficientName, n=Inf)
        coef_table_rmna <- coef_table[-which(is.na(coef_table$ENTREZID)), ]
        coef_logFC <- as.matrix(coef_table_rmna$logFC)
        row.names(coef_logFC) <- coef_table_rmna$ENTREZID

        for (pathway in pathwayList) {
            coef_path <- pathview(gene.data=coef_logFC, pathway.id=pathway, species=species, 
                                  out.suffix=sprintf("%s", coefficientName, kegg.native=TRUE))
        }
    }, error=function(e){
        print(paste0("runPathview failed with message:", e))
    }, finally = {
        setwd(workingDir)
    })
}

globalRunPathview = function(coefficientName){
    runPathview(gModeratedVoomedFitForContrastsWithLogFcThreshold, gOutputDir, 
        gKeggPathwaysOfInterest, gPathviewOrganismName, coefficientName)
}

### SPIA

As described in the SPIA [user manual](https://www.bioconductor.org/packages/devel/bioc/vignettes/SPIA/inst/doc/SPIA.pdf), "This package implements the Signaling Pathway Impact Analysis (SPIA) algorithm described in
Tarca et al. (2009 and 2018) ([9](#Citations)), Khatri et al. (2007) and Draghici et al. (2007). SPIA uses the information from
a set of differentially expressed genes and their fold changes, as well as pathways topology in order
to assess the significance of the pathways in the condition under the study." SPIA searches and reports KEGG pathways. If SPIA finds significant pathways, the KEGG IDs of the interesting pathways can be further visualized using Pathview. 

<div class="alert alert-warning">
    <strong>Analyst Note: Modify Code</strong><br />
    
**Set the appropriate organism name recognized by `SPIA`.**

A comprehensive list of 3 letter abbreviations can be found at this website: https://www.genome.jp/kegg/catalog/org_list.html 

Commonly used organism abbreviations:
* Human = hsa
* Mouse = mmu

</div>

In [40]:
gSPIAOrganismName = "hsa"

In [41]:
runSPIA = function(efitObject, outputDir, coefficientName, species, maxAdjPVal=0.05){
    tryCatch({
        # Get list of DE genes sorted by p-value
        top <- topTreat(efitObject, coef = coefficientName, n = Inf, sort.by = "p")
        top$ENTREZ <- unlist(as.list(top$ENTREZID))
        top <- top[!is.na(top$ENTREZID),]
        top <- top[!duplicated(top$ENTREZ),]
        tg1 <- top[top$adj.P.Val<maxAdjPVal,]
        DE_genes <- tg1$logFC
        names(DE_genes) <- as.vector(tg1$ENTREZ)
        ALL_genes <- top$ENTREZ
        # Pathway analysis
        res <- spia(de=DE_genes, all=ALL_genes, organism=species, nB=2000, 
                    plots=FALSE, beta=NULL, combine="fisher", verbose=FALSE)
        # Write results
        write.csv(res, sprintf("%s/%s_SPIA_%s.csv", outputDir, gRunName, coefficientName))
    }, error=function(e){print(paste0("runSPIA failed for maxAdjPval <",maxAdjPVal," with message:", e))})
}

globalRunSPIA = function(coefficientName){
    runSPIA(gModeratedVoomedFitForContrastsWithLogFcThreshold, gOutputDir, coefficientName, gSPIAOrganismName, maxAdjPVal=0.05)
}

### GSVA
"The GSVA package ([10](#Citations)) allows one to perform a change in coordinate systems of molecular measurements, transforming the data from a gene by sample matrix to a gene-set by sample matrix, thereby allowing the evaluation of pathway enrichment for each sample. This new matrix of GSVA enrichment scores facilitates applying standard analytical methods like functional enrichment, survival analysis, clustering, CNV-pathway analysis or cross-tissue pathway analysis, in a pathway-centric manner." -R. Castelo (GSVA developer)

In this implementation of GSVA, we search for enrichment in the C2: curated gene sets collection (Broad Institute) from the Molecular Signatures Database (MSigDB). More information regarding the gene sets can be found [here](http://software.broadinstitute.org/gsea/msigdb/collections.jsp).

In [42]:
runGSVA = function(previousVoomResult, previousContrastMatrix, previousDesign, outputDir, coefficientName, maxAdjPVal=0.05){
    tryCatch({
        # Load c2 Broad sets from KEGG, Reactome, and Biocarta
        canonicalC2BroadSets <- c2BroadSets[c(grep("^KEGG", names(c2BroadSets)), 
                                              grep("^REACTOME", names(c2BroadSets)),
                                              grep("^BIOCARTA", names(c2BroadSets)))]
        # Use Entrez IDs
        gene_exp <- previousVoomResult$E
        gene_ids <- previousVoomResult$genes
        row.names(gene_exp) <- gene_ids$ENTREZID
        
        gsva_exp <- gsva(gene_exp, c2BroadSets, mx.diff=TRUE, verbose=TRUE, parallel.sz=1, min.sz=5, max.sz=500)
        # Uncomment block= and correlation= in line below where applicable
        fit <- lmFit(gsva_exp, previousDesign)#, block=gPreviousStepEnv$gBlockVector, correlation=gPreviousStepEnv$gDupCorr$conensus) 
        fit2 <- contrasts.fit(fit, contrasts=previousContrastMatrix)
        fit2 <- eBayes(fit2)
        
        #Extract DE genes ranked by adj. p-value
        allGeneSets <- topTable(fit2, coef=coefficientName, number=Inf)
        DEGeneSets <- topTable(fit2, coef=coefficientName, number=Inf, p.value=maxAdjPVal, adjust="BH")
        res <- decideTests(fit2, p.value=maxAdjPVal)
        summary(res)
        write.csv(DEGeneSets, sprintf("%s/%s_GSEA_%s.csv", outputDir, gRunName, coefficientName))   
    }, error=function(e){print(paste0("runGSVA failed for maxAdjPval <",maxAdjPVal," with message:", e))})
}

globalRunGSVA = function(coefficientName){
    runGSVA(gPreviousStepEnv$gVoomResult, gPreviousStepEnv$gContrastMatrix, gPreviousStepEnv$gDesignModel, gOutputDir, coefficientName, maxAdjPVal=0.05)
}

[Table of Contents](#Table-of-Contents)

## Examination of Individual Comparisons

Perform both ORA and GSEA using WebGestalt against each of the annotated data sets specified and run pathview for each of the KEGG pathways specified.  Each comparison produces the following information:

* A single-column text file (named [runname]\_DE\_Genes\_Webgestalt\_ORA\_input\_[comparisonname].txt) containing the Entrez Gene ids of the differentially expressed genes for that comparison with FDR<0.05, which are used as the genes of interest in the ORA.
* A single-column text file (named [runname]\_DE\_Genes\_ReferenceGenes\_Webgestalt\_ORA\_[comparisonname].txt) containing the Entrez Gene ids of the reference genes used in the ORA, which is the complete set of genes for which differential expression testing was done.
* For each tested gene set, a folder (named Project_[runname]\_Webgestalt\_ORA\_[comparisonname]\_[annotatedsetname]), each of which contains an html report (named Report\_[runname]\_Webgestalt\_ORA\_[comparisonname]\_[annotatedsetname].html) describing the results of the ORA for the differentially expressed genes for that comparison against that annotated gene set.
* A two-column text file (named [runname]\_DE\_GenesAndScores\_Webgestalt\_GSEA\_input\_[comparisonname].rnk) containing the Entrez Gene ids of the differentially expressed genes for that comparison with FDR<0.05 and their log fold-change values.
* For each tested gene set, a folder (named Project\_[runname]\_Webgestalt\_GSEA\_SevereD5vsSevereD1\_[comparisonname]\_[annotatedsetname]), each of which contains an html report (named Report\_[runname]\_Webgestalt\_GSEA\_SevereD5vsSevereD1\_[comparisonname]\_[annotatedsetname]) describing the results of the GSEA for the ranked differentially expressed genes for that comparison against that annotated gene set.
* A folder (named Pathview_[comparisonname]) containing one file name [keggpathwayname].[comparisonname].png for each KEGG pathway tested.  Note that this folder will also contain other .png and .xml files, but are supporting files for pathview that can be ignored.

In [27]:
colnames(coefficients(gModeratedVoomedFitForContrastsWithLogFcThreshold))

Loading required package: limma

Attaching package: ‘limma’

The following object is masked from ‘package:BiocGenerics’:

    plotMA



[1] "Day15vsDay0"  "Day25vsDay0"  "Day25vsDay15" "Day25vsDay20"

<div class="alert alert-warning">
    <strong>Analyst Note: Info</strong><br />

In the next cell, note that the contrast name is intentionally NOT set as a global variable; **please don't change this**.  Setting the contrast name explicitly in each call prevents misleading records due to inconsistent state.
   
</div>

<div class="alert alert-warning">
    <strong>Analyst Note: Modify Code</strong><br />
    
**For each contrast of interest**, chosen from the list produced by running the cell above: 

* Add a markdown cell containing a link back to the table of contents and the name of the contrast as a 4th-level header (e.g:

        [Table of Contents](#Table-of-Contents)
        
        #### SevereD1vsMildD1
        
* Add a link to this cell in the Table of Contents markdown (e.g., `* [SevereD1vsMildD1](#SevereD1vsMildD1)`)
* Add a code cell running `globalRunWebgestalt(x)` where `x` is the contrast name
* If you want to run Pathview, add a code cell running `globalRunPathview(x)` where `x` is the contrast name
* If you want to run SPIA, add a code cell running `globalRunSPIA(x)` where `x` is the contrast name
* If you want to run GSVA, add a code cell running `globalRunGSVA(x)` where `x` is the contrast name
   
</div>

In [28]:
globalRunWebgestalt('Day15vsDay0')

Loading the functional categories...
Loading the ID list...
Loading the reference list...
Summarizing the input ID list by GO Slim data...
Performing the enrichment analysis...
Begin affinity propagation...
End affinity propagation...
Begin weighted set cover...
Remain is 0, ending weighted set cover
Generate the final report...
Results can be found in the ../outputs/Project_20190307_doe_human_patient_timeseries_enrichment_analyses_20191101143736_Webgestalt_ORA_Day15vsDay0_pathway_Panther!
Loading the functional categories...
Loading the ID list...
Loading the reference list...
Summarizing the input ID list by GO Slim data...
Performing the enrichment analysis...
Begin affinity propagation...
End affinity propagation...
Begin weighted set cover...
End weighted set cover...
Generate the final report...
Results can be found in the ../outputs/Project_20190307_doe_human_patient_timeseries_enrichment_analyses_20191101143736_Webgestalt_ORA_Day15vsDay0_disease_OMIM!
Loading the functional cat

Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for P02785
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for P02748
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for P00015
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for P05728
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for P02770
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for P02787
”Warning message 

”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for P02726
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for P04384
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for P02781
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for P02741
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for P02736
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for P02780
”Warning message

1000 permutations of score complete...
Begin affinity propagation...
End affinity propagation...
Begin weighted set cover...
Remain is 0, ending weighted set cover
Generate the final report...
Results can be found in the ../outputs/Project_20190307_doe_human_patient_timeseries_enrichment_analyses_20191101143736_Webgestalt_GSEA_Day15vsDay0_pathway_Panther!
Loading the functional categories...
Loading the ID list...
Summarizing the uploaded ID list by GO Slim data...
Performing the enrichment analysis...


Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for 535000
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for 540000
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for 500008
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for 551500
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for 116600
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for 610131
”Warning message 

...

”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB14042
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB14050
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB14059
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB14132
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB14146
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB14185
”Warning m

”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB00718
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB00787
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB00833
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB00856
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB00878
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB00958
”Warning m

”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB09342
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB09344
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB09374
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB09395
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB09459
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB09498
”Warning m

”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB06215
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB06784
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB08795
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB08830
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB08831
”Warning message in swGsea(inputDf, thresh_type = "val", perms = perNum, min_set_size = minNum, :
“score does not contain minimum number of items in set for DB08834
”Warning m

1000 permutations of score complete...
Begin affinity propagation...
End affinity propagation...
Begin weighted set cover...
Remain is 0, ending weighted set cover
Generate the final report...
Results can be found in the ../outputs/Project_20190307_doe_human_patient_timeseries_enrichment_analyses_20191101143736_Webgestalt_GSEA_Day15vsDay0_drug_DrugBank!


In [43]:
globalRunPathview('Day15vsDay0')

Warning message in dir.create(pathviewOutputDir):
“'../outputs/Pathview_Day15vsDay0' already exists”'select()' returned 1:1 mapping between keys and columns
Info: Working in directory /Users/amandabirmingham/Work/Repositories/rnaseq/outputs/Pathview_Day15vsDay0
Info: Writing image file hsa04512.Day15vsDay0.png
Warning message in structure(x$children, class = "XMLNodeList"):
“Calling 'structure(NULL, *)' is deprecated, as NULL cannot have attributes.
  Consider 'structure(list(), *)' instead.”Warning message in structure(x$children, class = "XMLNodeList"):
“Calling 'structure(NULL, *)' is deprecated, as NULL cannot have attributes.
  Consider 'structure(list(), *)' instead.”Warning message in structure(x$children, class = "XMLNodeList"):
“Calling 'structure(NULL, *)' is deprecated, as NULL cannot have attributes.
  Consider 'structure(list(), *)' instead.”Warning message in structure(x$children, class = "XMLNodeList"):
“Calling 'structure(NULL, *)' is deprecated, as NULL cannot have att

  Consider 'structure(list(), *)' instead.”Warning message in structure(x$children, class = "XMLNodeList"):
“Calling 'structure(NULL, *)' is deprecated, as NULL cannot have attributes.
  Consider 'structure(list(), *)' instead.”Warning message in structure(x$children, class = "XMLNodeList"):
“Calling 'structure(NULL, *)' is deprecated, as NULL cannot have attributes.
  Consider 'structure(list(), *)' instead.”Warning message in structure(x$children, class = "XMLNodeList"):
“Calling 'structure(NULL, *)' is deprecated, as NULL cannot have attributes.
  Consider 'structure(list(), *)' instead.”Warning message in structure(x$children, class = "XMLNodeList"):
“Calling 'structure(NULL, *)' is deprecated, as NULL cannot have attributes.
  Consider 'structure(list(), *)' instead.”Warning message in structure(x$children, class = "XMLNodeList"):
“Calling 'structure(NULL, *)' is deprecated, as NULL cannot have attributes.
  Consider 'structure(list(), *)' instead.”Warning message in structure(x$c

In [44]:
globalRunSPIA('Day15vsDay0')

In [50]:
globalRunGSVA('Day15vsDay0')

Estimating GSVA scores for 2834 gene sets.
Computing observed enrichment scores
Estimating ECDFs with Gaussian kernels
Using parallel with 1 cores
  |======================================================================| 100%


[Table of Contents](#Table-of-Contents)
## Understanding the Output
The following section explains the columns in each output file. It also provides some links to user guides.

### WebGestalt

Webgestalt output is labeled with the method applied (ORA or GSEA) and the functional database searched (typically geneontology_Biological_Process, geneontology_Molecular_Function, etc.). Within each results folder, look for the "Report....html" file. 

Further information is available in the [WebGestalt user manual](http://www.webgestalt.org/WebGestalt_2019_Manual.pdf) .

### Pathview
Images of KEGG pathway(s) with gene expression (logFC) color coded. The scale (-1 to 1) is a pseudo-scale and doesn't actually represent the raw logFC value.

### SPIA

Column definitions

* pSize: the number of genes on the pathway
* NDE: the number of DE genes per pathway
* tA: observed total perturbation accumulation in the pathway
* pNDE: the probabilirty to observe at least NDE genes on the pathway using a hypergeometric model
* pPERT: the probability to observe a total accumulation more extreme than tA only by chance
* pG: the p-value obtained by combining pNDE and pPERT
* pGFdr: the False Discovery Rate 
* pGFWER: Bonferroni adjusted global p-values
* Status: the direction in which the pathway is perturbed (activated or inhibited)
* KEGGLINK: URL to the KEGG website that displays the pathway image with the differentially expressed genes highlighted

(Descriptions above are from p.6 of the [SPIA user manual](http://bioconductor.org/packages/release/bioc/manuals/SPIA/man/SPIA.pdf).)

### GSVA
GSVA results show enriched gene set names from MSigDB. The gene sets can be searched in Pubmed and/or Google to find the associated publication. Typically, the gene sets begin with the last name of the first author of the publication.

Column definitions

* logFC: the log2 fold change of the gene set 
* AveExpr: in logCPM
* t: t-statistic
* P.Value: self explanatory
* adj.P.Val: Benjamini-Hochberg adjusted p-value
* B: B-statistic

[Table of Contents](#Table-of-Contents)

Save the workspace objects for future reference:

In [ ]:
writeWorkspaceImage(gInterimDir, gRunName)

[Table of Contents](#Table-of-Contents)

## Summary

<div class="alert alert-warning">
    <strong>Analyst Note: Fill In</strong><br />
    
Replace this cell with a summary of key points from this notebook in this section. This summary should highlight important findings and draw conclusions for the researcher. 

</div>

[Table of Contents](#Table-of-Contents)

## Citations

1. Law CW, Alhamdoosh M, Su S, Smyth GK, Ritchie ME. RNA-seq analysis is easy as 1-2-3 with limma, Glimma and edgeR. Version 2. F1000Res. 2016 Jun 17 [revised 2016 Jan 1];5:1408.
2. Wang J, Vasaikar S, Shi Z, Greer M, Zhang B. WebGestalt 2017: a more comprehensive, powerful, flexible and interactive gene set enrichment analysis toolkit. Nucleic Acids Res. 2017 Jul 3;45(W1):W130-W137.
3. R Core Team (2016). R: A language and environment for statistical computing. R Foundation for Statistical Computing, Vienna, Austria. URL https://www.R-project.org/.
4. Luo, Weijun, Brouwer, Cory (2013). Pathview: an R/Bioconductor package for pathway-based data integration and visualization. Bioinformatics, 29(14), 1830-1831. doi: 10.1093/bioinformatics/btt285.
5. Huber W, Carey VJ, Gentleman R, Anders S, Carlson M, Carvalho BS, Bravo HC, Davis S, Gatto L, Girke T, Gottardo R, Hahne F, Hansen KD, Irizarry RA, Lawrence M, Love MI, MacDonald J, Obenchain V, Oleś AK, Pagès H, Reyes A, Shannon P, Smyth GK, Tenenbaum D, Waldron L, Morgan M. Orchestrating high-throughput genomic analysis with Bioconductor. Nat Methods. 2015 Feb;12(2):115-21.
6. Kanehisa M, Goto S, Kawashima S, Okuno Y, Hattori M. The KEGG resource for deciphering the genome. Nucleic Acids Res. 2004 Jan 1;32(Database issue):D277-80.
7. Ashburner M, Ball CA, Blake JA, Botstein D, Butler H, Cherry JM, Davis AP, Dolinski K, Dwight SS, Eppig JT, Harris MA, Hill DP, Issel-Tarver L, Kasarskis A, Lewis S, Matese JC, Richardson JE, Ringwald M, Rubin GM, Sherlock G. Gene ontology: tool for the unification of biology. The Gene Ontology Consortium. Nat Genet. 2000 May;25(1):25-9.
8. Subramanian A, Tamayo P, Mootha VK, Mukherjee S, Ebert BL, Gillette MA, Paulovich A, Pomeroy SL, Golub TR, Lander ES, Mesirov JP. Gene set enrichment analysis: a knowledge-based approach for interpreting genome-wide expression profiles. Proc Natl Acad Sci U S A. 2005 Oct 25;102(43):15545-50.
9. Tarca AL, Kathri P, Draghici S (2018). SPIA: Signaling Pathway Impact Analysis (SPIA) using combined evidence of pathway over-representation and unusual signaling perturbations. R package version 2.34.0, http://bioinformatics.oxfordjournals.org/cgi/reprint/btn577v1.
10. Hänzelmann S., Castelo R. and Guinney J. GSVA: gene set variation analysis for microarray and RNA-Seq data. BMC Bioinformatics, 14:7, 2013.

[Table of Contents](#Table-of-Contents)

## Appendix: R Session Info

In [ ]:
sessionInfo()

[Table of Contents](#Table-of-Contents)

Copyright (c) 2018 UC San Diego Center for Computational Biology & Bioinformatics under the MIT License

Notebook template by Amanda Birmingham